In [1]:
import vrplib
import numpy as np
import pandas as pd

In [2]:
from datetime import datetime
from copy import deepcopy
from typing import List
from tqdm import tqdm
import collections


class Annealing:
  def __init__(self, instance,
              epochs = 100,
              k = 0.9,
              max_iter = 100,
               theta = 100 ):
    self.epochs = epochs
    self.k = k
    self.max_iter = max_iter
    self.theta = theta
    self.instance = instance
    #self.nodes = instance['node_coord']        # координаты города
    self.demand = instance['demand']           # требования
    self.edge_weight = instance['edge_weight'] # расстояния
    self.trucks = int(np.ceil(np.sum(instance['demand']) / instance['capacity'])) # cколько нужно / на сколько могут возить

  def init_routes(self): # random
    
    routes = []
    free_nodes = [t for t in range(self.instance['dimension']) ]
    free_nodes.remove(0)

    for _ in range(self.trucks):
        cur_capacity = self.instance['capacity']
        idx = np.random.choice(list(free_nodes))
        route = []

        while (self.demand[idx] <= cur_capacity):
            cur_capacity -= self.demand[idx]
            free_nodes.remove(idx)
            route.append(idx)

            if len(free_nodes) == 0:
              break
            else:
              idx = np.random.choice(free_nodes)            
        
        routes.append(route)
    for i in range(len(routes)):
        routes[i] = [0] + routes[i] + [0]
    return routes
  
  def target_function(self, routes):
    sums = 0
    for truck_routes in routes:
        for i in range(len(truck_routes)-1):
          sums += self.edge_weight[truck_routes[i], truck_routes[i+1]]

    return sums

  def shuffle_order(self, routes):
    new_routes = deepcopy(routes)

    for i in range(len(new_routes)):
        if len(new_routes[i]) <= 3:
            continue
        
        idx_1 = np.random.randint(1,len(new_routes[i])-1)
        idx_2 = np.random.randint(1,len(new_routes[i])-1)
        while idx_1 == idx_2:
            idx_2 = np.random.randint(1,len(new_routes[i])-1)
        
        new_routes[i][idx_1], new_routes[i][idx_2] = new_routes[i][idx_2], new_routes[i][idx_1]
    return new_routes

  def exchange_verticies(self, routes):
    new_routes = []

    while not self.is_valid_routes(new_routes) or self.is_same_routes(routes, new_routes):
        new_routes = deepcopy(routes)
        route_idx_1 = np.random.randint(0,len(new_routes))
        route_idx_2 = np.random.randint(0,len(new_routes))
        while route_idx_1 == route_idx_2 or len(new_routes[route_idx_1]) < 3 or len(new_routes[route_idx_2]) < 3:
            route_idx_1 = np.random.randint(0,len(new_routes))
            route_idx_2 = np.random.randint(0,len(new_routes))
        if len(new_routes[route_idx_1])-1 == 1:
                random_idx_1 = 1
        else:
                random_idx_1 = np.random.randint(1,len(new_routes[route_idx_1])-1)
        if len(new_routes[route_idx_2])-1 == 1:
                random_idx_2 = 1
        else:
                random_idx_2 = np.random.randint(1,len(new_routes[route_idx_2])-1)
        new_routes[route_idx_1][random_idx_1], new_routes[route_idx_2][random_idx_2] = new_routes[route_idx_2][random_idx_2], new_routes[route_idx_1][random_idx_1]

    return new_routes

  def shuffle_verticies(self, routes):
    new_routes = []

    while not self.is_valid_routes(new_routes) or self.is_same_routes(routes, new_routes):
        new_routes = deepcopy(routes)
        route_idx_1 = np.random.randint(0,len(new_routes))
        route_idx_2 = np.random.randint(0,len(new_routes))
        while route_idx_1 == route_idx_2 or len(new_routes[route_idx_1]) < 3 or len(new_routes[route_idx_2]) < 3:
            route_idx_1 = np.random.randint(0,len(new_routes))
            route_idx_2 = np.random.randint(0,len(new_routes))
        
        if len(new_routes[route_idx_1])-1 == 1:
            random_idx_1 = 1
        else:
            random_idx_1 = np.random.randint(1,len(new_routes[route_idx_1])-1)
            
        random_idx_2 = np.random.randint(1,len(new_routes[route_idx_2]))
        new_routes[route_idx_2] = new_routes[route_idx_2][:random_idx_2] + [new_routes[route_idx_1][random_idx_1]] + new_routes[route_idx_2][random_idx_2:] 
        del new_routes[route_idx_1][random_idx_1]

    return new_routes

  def get_new_sol(self, routes):
    transformed = False
    while not transformed:
        if (np.random.random() <= 0.5):
            routes = self.shuffle_order(routes)
            transformed = True
        if (np.random.random() <= 0.5):
            routes = self.shuffle_verticies(routes)
            transformed = True
        if (np.random.random() <= 0.5):
            routes = self.exchange_verticies(routes)
            transformed = True

    return routes

  def is_same_routes(self, routes_1, routes_2):
    if len(routes_1) != len(routes_2):
        return False
    compare = lambda x, y: collections.Counter(x) == collections.Counter(y)

    checked_idxs_i = set()
    checked_idxs_j = set()
    for i in range(len(routes_1)):
        for j in range(len(routes_2)):
            if i in checked_idxs_i or j in checked_idxs_j:
                continue
            
            if compare(routes_1[i], routes_2[j]):
                checked_idxs_i.add(i)
                checked_idxs_j.add(j)
            else:
                if j == len(routes_2) - 1:
                    return False
    return True

  def is_valid_routes(self, routes):
    if len(routes) == 0:
        return False

    for route in routes:
        route_demand = 0
        for idx in route:
            route_demand += self.demand[idx]

        if route_demand > self.instance['capacity']:
            return False
    return True

  def fit(self):
    old_solution = self.init_routes()
    loss = []
    min_loss = np.inf
    epochs = self.epochs
    k = self.k         #  Коэффициент понижения температуры 
    max_iter = self.max_iter
    time1 = datetime.now()
    for _ in range(epochs):
        theta = self.theta

        while theta > 1:
            #print(old_solution)
            for i in range(max_iter):
              new_solution = self.get_new_sol(old_solution)
              prob = np.exp(-(self.target_function(new_solution) - self.target_function(old_solution))/theta)   
              if self.target_function(new_solution) - self.target_function(old_solution) < 0:
                  old_solution = new_solution
              elif prob >= np.random.random():    
                  old_solution = new_solution
            theta *= k                            
       
        epoch_loss = self.target_function(old_solution)
        loss.append(epoch_loss)

        if (epoch_loss < min_loss):
          min_loss = epoch_loss
          min_solution = old_solution
    
    time2 = datetime.now()
    time = time2 - time1
    return time, loss, min_solution

In [3]:
sample_list = list(
    filter(
        lambda x : (str(x[0]) == 'E') and str(x[1]) == '-',
        vrplib.list_names(vrp_type="cvrp")
        )
    )

In [4]:
df_dict = {'sample': [], 'time' : [], 'mape' : [], 'dimension' : []}
for sample in tqdm(sample_list):
    vrplib.download_instance(sample, f"{sample}.vrp")
    vrplib.download_solution(sample, f"{sample}.sol")

    instance = vrplib.read_instance(f"{sample}.vrp")
    solution = vrplib.read_solution(f"{sample}.sol")
    algo_annealing = Annealing(instance,
                              epochs = 200,
                              k = 0.9,
                              max_iter = 200,
                              theta = 500 )

    time, loss, min_solution = algo_annealing.fit()

    df_dict['sample'].append(sample)
    df_dict['time'].append(time)
    mape = (abs(solution['cost'] - min(loss)) / solution['cost'])*100
    df_dict['mape'].append(mape)

    df_dict['dimension'].append(instance['dimension'])

100%|██████████████████████████████████████████████████████████████████████████████| 13/13 [5:40:48<00:00, 1572.98s/it]


In [16]:
df = pd.DataFrame(df_dict)
df.to_csv("cvrp_E.csv", index=False)

In [17]:
df

,sample,time,mape,dimension
0,E-n13-k4,0 days 00:07:30.433199,17.408907,13
1,E-n22-k4,0 days 00:11:24.461825,2.400823,22
2,E-n23-k3,0 days 00:07:06.007714,25.002604,23
3,E-n30-k3,0 days 00:12:40.854774,7.648093,30
4,E-n31-k7,0 days 00:21:21.406938,192.348285,31
5,E-n33-k4,0 days 00:12:01.463822,2.867577,33
6,E-n51-k5,0 days 00:16:04.735235,24.110305,51
7,E-n76-k7,0 days 00:20:12.787490,53.526925,76
8,E-n76-k8,0 days 00:27:14.665973,46.727560,76
9,E-n76-k10,0 days 00:38:46.838440,31.476897,76


In [10]:
import matplotlib.pyplot as plt